
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

def normalize_data_min_max(input):
    min_original = np.min(input)
    max_original = np.max(input)
    input_normalizado = (input - min_original) / (max_original - min_original)
    return input_normalizado

def desnormalize_dados_min_max(output):
    min_original = np.min(output)
    max_original = np.max(output)
    dados_desnormalizados = output * (max_original - min_original) + min_original
    return dados_desnormalizados



/tmp/ipykernel_1272332/831156827.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-04-01 23:08:57.483630: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-01 23:08:57.528479: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 23:08:57.528519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 23:08:57.529769: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-01 23:08:57.535651: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-01 23:08:57.536191: I tensorflow/core/platform/cpu_feature_guard.cc:1

2024-04-01 23:08:58.767394: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    
    
    def create_dataset(self, input, output):           #imagino q seja aq q coloca o input normalizado
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      input_train_normalize= normalize_data_min_max(input_train)
      input_vt_normalize= normalize_data_min_max(input_vt)
      input_test_normalize= normalize_data_min_max(input_test)
      input_val_normalize= normalize_data_min_max(input_val)
      input_normalize= normalize_data_min_max(input)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train_normalize, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val_normalize, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test_normalize, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt_normalize, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input_normalize, output)
      
      self._train = (input_train_normalize, output_train)
      self._vt = (input_vt_normalize, output_vt)
      self._val = (input_val_normalize, output_val)
      self._test = (input_test_normalize, output_test)
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
        
      input_vt_normalize= normalize_data_min_max(self.vt_input)
      input_test_normalize= normalize_data_min_max(self.test_input)
      input_val_normalize= normalize_data_min_max(self.val_input)
      input_normalize= normalize_data_min_max(self.input)

      self.test_prediction = self.lm_model.predict(input_test_normalize)

      pred = self.lm_model.predict(input_normalize).flatten()
      test_pred = self.test_prediction.flatten()
      val_pred = self.lm_model.predict(input_val_normalize).flatten()
      vt_pred = self.lm_model.predict(input_vt_normalize).flatten()
          
      pred_desnormalizado = desnormalize_dados_min_max(pred)
      test_pred_desnormalizado = desnormalize_dados_min_max(test_pred)
      val_pred_desnormalizado = desnormalize_dados_min_max(val_pred)
      vt_pred_desnormalizado = desnormalize_dados_min_max(vt_pred)

      r2 = r2_score(self.output, pred_desnormalizado)
      r2_test = r2_score(self.test_output, test_pred_desnormalizado)
      r2_val = r2_score(self.val_output, val_pred_desnormalizado)
      r2_vt = r2_score(self.vt_output, vt_pred_desnormalizado)

      mse = mean_squared_error(self.output, pred_desnormalizado)
      mse_test = mean_squared_error(self.test_output, test_pred_desnormalizado)
      mse_val = mean_squared_error(self.val_output, val_pred_desnormalizado)
      mse_vt = mean_squared_error(self.vt_output, vt_pred_desnormalizado)

      metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }
      return metrics

In [6]:
from itertools import product
import pickle


class Tester:
  
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:

tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [9]:
tester.Loop(sort_by='r2',
            boundarie=0.5,
            hidden_sizes = [[45],[25], [30], [35], [40]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[45], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  0.4374682899997424
Stopped at epoch:  8
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.4228117419988848
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.2611665610002092
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.19833872500021243
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.1953783030003251
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.22541131199977826
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 7 +++++++++++

2024-04-01 23:44:12.214125: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.2404825929988874
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [19_0] | 17 ++++++++++++++++++
Elapsed time:  0.20700445900001796
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [19_0] | 18 ++++++++++++++++++
Elapsed time:  0.2405635840004834
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [19_0] | 19 ++++++++++++++++++
Elapsed time:  0.18994076500166557
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [19_0] | 20 ++++++++++++++++++
Elapsed time:  0.2116799980030919
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [19_0] | 21 ++++++++++++++++++
Elapsed time:  0.21332349599833833
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [19_0] | 22 ++++++++++++++++++
Elapsed time:  0.2608361740021792
Stopped at epoch:  6
1/1 [==============================] - 0s 